In [41]:
import boto3
import joblib
import os
import numpy as np
from sklearn.metrics import accuracy_score
import pandas as pd
from sagemaker.session import Session
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.session import get_execution_role
from sagemaker.tuner import HyperparameterTuner, CategoricalParameter, IntegerParameter, ContinuousParameter
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost.estimator import XGBoost
from sagemaker import image_uris
from sagemaker.inputs import TrainingInput
from sagemaker.estimator import Estimator
from sagemaker.serializers import CSVSerializer
import sagemaker

In [42]:
role = get_execution_role()
region = boto3.Session().region_name
bucket = Session().default_bucket()

In [63]:
container = image_uris.retrieve('xgboost', Session().boto_region_name, '1.2-1' )

In [44]:
s3_input_train = TrainingInput(s3_data='s3://{}/data/train-tf-44898-5000-1'.format(bucket), content_type='csv')
s3_input_test = TrainingInput(s3_data='s3://{}/data/test-tf-44898-5000-1'.format(bucket), content_type='csv')

In [33]:
xgb = Estimator(container,
                role,
                instance_count=1,
                instance_type='ml.m4.xlarge')

xgb.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='binary:logistic',
                        num_round=100)

xgb.fit({'train': s3_input_train, 'validation': s3_input_test})

2021-01-01 19:07:52 Starting - Starting the training job...
2021-01-01 19:08:15 Starting - Launching requested ML instancesProfilerReport-1609528072: InProgress
......
2021-01-01 19:09:22 Starting - Preparing the instances for training............
2021-01-01 19:11:17 Downloading - Downloading input data...
2021-01-01 19:11:49 Training - Training image download completed. Training in progress..INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Single node training.
IN

In [34]:
xgb_predictor = xgb.deploy(initial_instance_count=1,
                           instance_type='ml.m4.xlarge')

-----------------!

In [38]:
xgb_predictor.serializer = CSVSerializer()

In [103]:
validation_data = pd.read_csv('s3://{}/data/validation-tf-44898-5000-1/validation.csv'.format(bucket))
validation_y = validation_data.iloc[:, 0]
validation_X = validation_data.iloc[:, 1:]

In [20]:
xx = np.array_split(validation_X, 100)

In [41]:
def predict(data, rows=100):
    split_array = np.array_split(data, int(data.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, xgb_predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=',')

In [42]:
predictions = predict(validation_X.to_numpy())

In [43]:
pred_y = predictions.round()

In [44]:
accuracy_score(pred_y, validation_y)

0.9835750032080072

In [31]:
accuracy_score(pred_y, validation_y)

0.9708712947517002

In [45]:
xgb = sagemaker.estimator.Estimator(container,
                                    role, 
                                    instance_count=1, 
                                    instance_type='ml.c5.2xlarge',
                                    sagemaker_session=Session())

In [46]:
xgb.set_hyperparameters(objective='binary:logistic',
                        num_round=100,
                        rate_drop=0.3,
                        tweedie_variance_power=1.4)

In [47]:
hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': ContinuousParameter(1, 10),
                        'alpha': ContinuousParameter(0, 2),
                        'max_depth': IntegerParameter(1, 10)}

In [48]:
tuner = HyperparameterTuner(xgb,
                            objective_metric_name = 'validation:rmse', # The metric used to compare trained models.
                            objective_type = 'Minimize', # Whether we wish to minimize or maximize the metric.
                            hyperparameter_ranges=hyperparameter_ranges,
                            max_jobs=8,
                            max_parallel_jobs=8)

In [49]:
tuner.fit({'train': s3_input_train, 'validation': s3_input_test})

...............................................................................!


In [68]:
def perform_tuning(session, role, train_data, test_data, base_dict, sweep_dict, n_jobs=10, parallel_jobs=3):
    
    print(session)
    
    container = image_uris.retrieve('xgboost', session.boto_region_name, '1.2-1')
    
    print('here')
    
    regressor = sagemaker.estimator.Estimator(
        container,
        role, 
        instance_count=1, 
        instance_type='ml.c5.2xlarge',
        sagemaker_session=session
    ) 
    
    regressor.set_hyperparameters(**base_dict)
    
    tuner = HyperparameterTuner(
        regressor,
        objective_metric_name='validation:rmse',
        objective_type='Minimize',
        hyperparameter_ranges=sweep_dict,
        max_jobs=n_jobs,
        max_parallel_jobs=parallel_jobs
    )
    
    tuner.fit({'train': train_data, 'validation': test_data})
    
    #get_hyperparameters = tuner.hyperparameters()
    
    return tuner

In [89]:
def deploy_best_model(tuner):
    
    predictor = tuner.deploy(initial_instance_count=1, instance_type='ml.t2.xlarge')
    
    return predictor

In [109]:
def perform_prediction(predictor, validation_data, rows=100):
    
    predictor.serializer = CSVSerializer()
    
    validation_y = validation_data.iloc[:, 0]
    validation_X = validation_data.iloc[:, 1:].to_numpy()
    
    split_array = np.array_split(validation_X, int(validation_X.shape[0] / float(rows) + 1))
    predictions = ''
    for array in split_array:
        predictions = ','.join([predictions, predictor.predict(array).decode('utf-8')])

    return np.fromstring(predictions[1:], sep=','), validation_y.to_numpy()

In [ ]:
def validate_data(model, data_prefix, dataset_list, base_dict, sweep_dict, n_jobs=10, parallel_jobs=3):
    
    print('define some SageMaker base parameters...', end='')
        
    sagemaker_session = Session()
    role = get_execution_role()
    bucket = sagemaker_session.default_bucket()
    
    print('done')
    
    for dataset in dataset_list:
        
        model_name = '{}-{}'.format(model, dataset)
        print('evaluate model {}...'.format(model_name))
        
        # define input data
        input_data = []
        data_type_list = ['train', 'test', 'validation']
        for data_type in data_type_list:
            input_data.append('s3://{}/{}/{}-{}'.format(default_bucket, data_prefix, data_type, dataset))
        
        # configure hyperparameter tuning
        s3_input_train = TrainingInput(s3_data=input_data[0], content_type='csv')
        s3_input_test = TrainingInput(s3_data=input_data[1], content_type='csv')
        
        # start hyperparameter tuning job
        print('start tuning', end='')
        tuner, hyperparameters = perform_tuning(
            session=sagemaker_session, 
            role=role, 
            train_data=s3_input_train, 
            test_data=s3_input_test,
            base_dict=base_dict, 
            sweep_dict=sweep_dict, 
            n_jobs=n_jobs, 
            parallel_jobs=parallel_jobs
        )

        # deploy endpoint
        print('deploy best model', end='')
        predictor = deploy_best_model(tuner)
        print('')

        # read validation data
        validation_data = pd.read_csv('{}/{}'.format(input_data[2], 'validation.csv'))
        
        # read test data
        test_data = pd.read_csv('{}/{}'.format(input_data[1], 'test.csv'))
        
        # we have a lot of validation data, so we'll split it into batches of 100
        # split the validate data set into batches and evaluate using prediction endpoint  
        test_pred_y = perform_prediction(predictor, test_data, rows=100)
        validation_pred_y = perform_prediction(predictor, test_data, rows=100)              

        # get accuracy metrics
        test_accuracy = accuracy_score(test_y, test_pred_y)
        print('test model...accuracy: {} %'.format(round(test_accuracy * 100, 1)))
        validation_accuracy = accuracy_score(validation_y, validation_pred_y)
        print('validate model...accuracy: {} %'.format(round(validation_accuracy * 100, 1)))
        
        # save model results continuously into files in case something crashes, we have at least old results
        validation_prefix = 'validation-{}'.format(model) # add prefix where data will be stored
        tuned_model_prefix = 'tuned-model-{}'.format(model) # add prefix where best model data will be stored
        save_model_results(validation_accuracy, test_accuracy, dataset, validation_prefix)
        save_best_model_parameters(best_model_parameters, dataset, tuned_model_prefix)
        
        # remove resources
        best_predictor.delete_endpoint()

In [ ]:
validate_data('knn', 'data', data, base, ranges, jobs, parallel_jobs)

In [91]:
base_dict = {
    'objective': 'binary:logistic',
    'num_round': 100,
    'rate_drop':0.3,
    'tweedie_variance_power': 1.4
}

In [92]:
sweep_dict = {
    'eta': ContinuousParameter(0, 1),
    'min_child_weight': ContinuousParameter(1, 10),
    'alpha': ContinuousParameter(0, 2),
    'max_depth': IntegerParameter(1, 10)
}

In [93]:
sagemaker_session = Session()
role = get_execution_role()
bucket = sagemaker_session.default_bucket()

In [94]:
s3_input_train = TrainingInput(s3_data='s3://{}/data/train-tf-44898-5000-1'.format(bucket), content_type='csv')
s3_input_test = TrainingInput(s3_data='s3://{}/data/test-tf-44898-5000-1'.format(bucket), content_type='csv')
validation_data = pd.read_csv('s3://{}/data/validation-tf-44898-5000-1/validation.csv'.format(bucket))

In [95]:
tuner = perform_tuning(sagemaker_session, role, s3_input_train, s3_input_test, base_dict, sweep_dict, n_jobs=3, parallel_jobs=3)

here
...........................................................................!


In [96]:
predictor = deploy_best_model(tuner)


2021-01-01 21:40:47 Starting - Preparing the instances for training
2021-01-01 21:40:47 Downloading - Downloading input data
2021-01-01 21:40:47 Training - Training image download completed. Training in progress.
2021-01-01 21:40:47 Uploading - Uploading generated training model
2021-01-01 21:40:47 Completed - Training job completed
-----------------!

In [101]:
result = perform_prediction(predictor, validation_data, rows=100)

In [104]:
pred_y = result.round()

In [105]:
accuracy_score(pred_y, validation_y)

0.982676761195945

In [106]:
result

array([6.56565535e-04, 7.88106769e-03, 9.99999285e-01, ...,
       6.60055521e-05, 2.08484598e-05, 1.96906467e-06])

In [108]:
validation_y.to_numpy()

array([0, 0, 1, ..., 0, 0, 0])